In [1]:
province_csv = "R09_PI.csv"

In [2]:
import pandas as pd
import networkx as nx
import pickle

df = pd.read_csv("Data/comuni.csv")
df2 = pd.read_csv(f"Data/{province_csv}", sep=';')
df3 = pd.read_csv("Data/italy_geo.csv")

In [3]:
df2 = df2[df2["DEST_PROV"] == df2["OR_PROV"]]
province_code = df2["DEST_PROV"].iloc[0]

In [4]:
df = df[df['provincia/codice'] == province_code]

In [5]:
df2 = df2.drop(columns = ['OR_DEST', 'OR_REG', 'OR_PROV', 'DEST_REG', 'DEST_PROV', 'TEP_TOT', 'TTP_TOT '])
df2 = df2.replace(',', '.', regex=True)
df2['KM_TOT'] = df2['KM_TOT'].astype(float)
df2 = df2[df2['KM_TOT'] > 0]
df = df[['nome', 'codice', 'popolazione']]

In [6]:
df2['OR_PROCOM'] = df2['OR_PROCOM'].map(df.set_index('codice')['nome'])
df2['DEST_PROCOM'] = df2['DEST_PROCOM'].map(df.set_index('codice')['nome'])
df = df.drop(columns = ['codice'])
df3 = df3.drop(columns = ['istat'])
df3 = df3.drop_duplicates(subset = ['comune'])

In [8]:
G = nx.from_pandas_edgelist(df2, "OR_PROCOM", "DEST_PROCOM", "KM_TOT")
population_attr = df.set_index('nome').to_dict(orient = 'index')
position_attr = df3.set_index('comune').to_dict(orient = 'index')
nx.set_node_attributes(G, population_attr)
nx.set_node_attributes(G, position_attr)
for node in enumerate(G.nodes(data = True)):
    print(node)

(0, ('Bientina', {'popolazione': 7766, 'lng': 10.61983755, 'lat': 43.71011358}))
(1, ('Calcinaia', {'popolazione': 11684, 'lng': 10.61566661, 'lat': 43.68299475}))
(2, ('Vicopisano', {'popolazione': 8479, 'lng': 10.58439105, 'lat': 43.69975486}))
(3, ('Buti', {'popolazione': 5773, 'lng': 10.58735644, 'lat': 43.72731222}))
(4, ('Santa Maria a Monte', {'popolazione': 12847, 'lng': 10.6923682, 'lat': 43.69866516}))
(5, ('Pontedera', {'popolazione': 28061, 'lng': 10.63600988, 'lat': 43.66392507}))
(6, ('Cascina', {'popolazione': 43833, 'lng': 10.54927706, 'lat': 43.67654137}))
(7, ('Castelfranco di Sotto', {'popolazione': 12904, 'lng': 10.74523981, 'lat': 43.69969607}))
(8, ('Ponsacco', {'popolazione': 15237, 'lng': 10.63360648, 'lat': 43.62021159}))
(9, ("Montopoli in Val d'Arno", {'popolazione': 11167, 'lng': 10.76042631, 'lat': 43.66858781}))
(10, ('Calci', {'popolazione': 6409, 'lng': 10.51568636, 'lat': 43.72555225}))
(11, ("Santa Croce sull'Arno", {'popolazione': 14061, 'lng': 10.778

In [8]:
with open('distances.gpickle', 'wb') as f:
    pickle.dump(G, f, pickle.HIGHEST_PROTOCOL)